In [9]:
import numpy as np
from pgmpy.models import MarkovNetwork
from pgmpy.factors.discrete import DiscreteFactor
from pgmpy.inference.ExactInference import BeliefPropagation

width, height = 5, 5
np.random.seed(42)
original = np.random.randint(0, 2, (height, width))

noisy = original.copy()
noise_mask = np.random.rand(height, width) < 0.1
noisy[noise_mask] = 1 - noisy[noise_mask]

model = MarkovNetwork()
nodes = [(i, j) for i in range(height) for j in range(width)]
model.add_nodes_from(nodes)

edges = []
for i in range(height):
    for j in range(width):
        if i + 1 < height:
            edges.append(((i, j), (i + 1, j)))
        if j + 1 < width:
            edges.append(((i, j), (i, j + 1)))
model.add_edges_from(edges)

lam = 2.0
pixel_values = [0, 1]

factors = []
for i in range(height):
    for j in range(width):
        y_i = noisy[i, j]
        phi_unary_values = [np.exp(-lam * (x_i - y_i)**2) for x_i in pixel_values]
        phi_unary = DiscreteFactor(variables=[(i, j)], cardinality=[2], values=phi_unary_values)
        factors.append(phi_unary)

for edge in edges:
    phi_pairwise_values = []
    for x_i in pixel_values:
        for x_j in pixel_values:
            phi_pairwise_values.append(np.exp(-(x_i - x_j)**2))
    phi_pairwise = DiscreteFactor(variables=[edge[0], edge[1]], cardinality=[2, 2], values=phi_pairwise_values)
    factors.append(phi_pairwise)

model.add_factors(*factors)

bp = BeliefPropagation(model)
map_result = bp.map_query(variables=nodes)

denoised = np.zeros((height, width), dtype=int)
for (i, j), value in map_result.items():
    denoised[i, j] = value

print("Original Image:\n", original)
print("Noisy Image:\n", noisy)
print("Denoised Image:\n", denoised)

Original Image:
 [[0 1 0 0 0]
 [1 0 0 0 1]
 [0 0 0 0 1]
 [0 1 1 1 0]
 [1 0 1 1 1]]
Noisy Image:
 [[0 0 0 0 0]
 [0 1 0 0 0]
 [0 0 1 0 1]
 [0 1 1 1 0]
 [0 0 1 1 0]]
Denoised Image:
 [[0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 1 0 0]
 [0 0 1 1 0]
 [0 0 1 1 0]]
